In [ ]:
import numpy as pd
import pandas as pd
import re
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
class CFG:
    clean_before_matching = True
    
    
CLN_BFR_MTCH = CFG.clean_before_matching
if CLN_BFR_MTCH:
    LABEL = "cleaned_label"
else:
    LABEL = "dataset_label"

In [ ]:
train = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
train

In [ ]:
dataset_label_train = list(set(train[LABEL]))
del train

In [ ]:
df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")

def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def get_text(filename, train=False):
    if train:
        df = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/train/{filename}.json')
    else:
        df = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/test/{filename}.json')

    text = " ".join(list(df['text']))
    
    if CLN_BFR_MTCH:
        text = clean_text(text)
    
    return text

df["text"] = df["Id"].progress_apply(lambda x: get_text(x))
df

In [ ]:
def string_matching(txt, labels):
    """
    Args:
        txt: str
        lables: List[str]
    Returns:
        preds: str
    """
    preds = []
    for label in labels:
        if label in txt:
            preds.append(label)
    
    if not CLN_BFR_MTCH:
        preds = [clean_text(pred) for pred in preds]
    
    preds = "|".join(preds)
    return preds

df["PredictionString"] = df["text"].progress_apply(lambda x: string_matching(x, dataset_label_train))
df.drop("text", axis=1, inplace=True)
df

In [ ]:
df.to_csv("submission.csv", index=False)